In [17]:
from paddleocr import PaddleOCR, draw_ocr # main OCR dependencies
from matplotlib import pyplot as plt # plot images
import cv2 #opencv
import os

In [28]:
ocr_model = PaddleOCR(use_gpu=False,lang='french')


[2024/03/03 18:40:17] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\kortb/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\kortb/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_te

In [20]:
os.environ["PATH"] += os.pathsep + "C:/Users/kortb/Desktop/BFI/myenv"


In [29]:
img_path = '3.png'
result = ocr_model.ocr(img_path, cls=True)

[2024/03/03 18:40:21] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/03/03 18:40:21] ppocr DEBUG: dt_boxes num : 49, elapsed : 0.11252665519714355
[2024/03/03 18:40:22] ppocr DEBUG: rec_res num  : 49, elapsed : 1.087775468826294


In [30]:
result = ocr_model.ocr(img_path)


[2024/03/03 18:40:24] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/03/03 18:40:24] ppocr DEBUG: dt_boxes num : 49, elapsed : 0.08952474594116211
[2024/03/03 18:40:25] ppocr DEBUG: rec_res num  : 49, elapsed : 0.9762678146362305


In [31]:
result

[[[[[19.0, 3.0], [52.0, 3.0], [52.0, 19.0], [19.0, 19.0]],
   ('PASSIE', 0.9941186308860779)],
  [[[1.0, 22.0], [194.0, 22.0], [194.0, 35.0], [1.0, 35.0]],
   ('1 - Banque centrale et Centre de cheques postaux', 0.9737885594367981)],
  [[[369.0, 23.0], [378.0, 23.0], [378.0, 36.0], [369.0, 36.0]],
   (' 0', 0.7499680519104004)],
  [[[2.0, 41.0], [242.0, 41.0], [242.0, 55.0], [2.0, 55.0]],
   ('2 - Dépöts et avoirs des etablissements bancaires et financiers',
    0.9691433906555176)],
  [[[294.0, 42.0], [311.0, 42.0], [311.0, 57.0], [294.0, 57.0]],
   ('4.7', 0.998425304889679)],
  [[[348.0, 42.0], [378.0, 42.0], [378.0, 57.0], [348.0, 57.0]],
   ('31 750', 0.9897408485412598)],
  [[[426.0, 42.0], [456.0, 42.0], [456.0, 57.0], [426.0, 57.0]],
   ('57 533', 0.9681021571159363)],
  [[[2.0, 71.0], [133.0, 71.0], [133.0, 85.0], [2.0, 85.0]],
   ('3 - Depots et avoirs de la clientele', 0.9898083806037903)],
  [[[294.0, 70.0], [312.0, 70.0], [312.0, 88.0], [294.0, 88.0]],
   ('4.8', 0.9892773

In [42]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
img = Image.open(img_path)

image = img.convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['decoder.output_projection.weight', 'encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
from transformers import TrOCRConfig, TrOCRForCausalLM

# Initializing a TrOCR-base style configuration
configuration = TrOCRConfig()

# Initializing a model (with random weights) from the TrOCR-base style configuration
model = TrOCRForCausalLM(configuration)

# Accessing the model configuration
configuration = model.config

In [ ]:
from transformers import (
    TrOCRConfig,
    TrOCRProcessor,
    TrOCRForCausalLM,
    ViTConfig,
    ViTModel,
    VisionEncoderDecoderModel,
)
import requests
from PIL import Image

# TrOCR is a decoder model and should be used within a VisionEncoderDecoderModel
# init vision2text model with random weights
encoder = ViTModel(ViTConfig())
decoder = TrOCRForCausalLM(TrOCRConfig())
model = VisionEncoderDecoderModel(encoder=encoder, decoder=decoder)

# If you want to start from the pretrained model, load the checkpoint with `VisionEncoderDecoderModel`
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
uimg = Image.open(img_path)

image = img.convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values
text = "industry, ' Mr. Brown commented icily. ' Let us have a"

# training
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

labels = processor.tokenizer(text, return_tensors="pt").input_ids
outputs = model(pixel_values, labels=labels)
loss = outputs.loss
round(loss.item(), 2)

# inference
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text